In [1]:
%load_ext blackcellmagic
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
from itertools import chain
import pandas as pd
from tabulate import tabulate
import os
import requests
from math import ceil
import traceback

In [2]:
def flatMap(function_to_apply, list_of_inputs, filter_to_apply=lambda x: True):
    return list(
        chain.from_iterable(
            [function_to_apply(el) for el in list_of_inputs if filter_to_apply(el)]
        )
    )


In [3]:
# create a new Chrome session
driver = webdriver.Chrome("../../chromedriver")
driver.implicitly_wait(30)

In [4]:
base_url = "https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12"

In [5]:
page = driver.get(base_url)

In [6]:
bs = BeautifulSoup(driver.page_source, "html.parser")

In [7]:
(total, particulier, pro) = tuple(map(lambda e: int(e.text), bs.find_all("span", class_="_2ilNG")))

In [8]:
total, particulier, pro

(211, 10, 201)

In [9]:
MAX_PER_PAGE = 35
particuliers_urls = []
num_pages = ceil(particulier / 35)

for num in range(1, num_pages+1):
    particuliers_tab = base_url + "&owner_type=private" + ("&page="+num if num > 1 else "")
    print(particuliers_tab)
    driver.get(particuliers_tab)
    bs = BeautifulSoup(driver.page_source, "html.parser")
    particuliers_urls.extend([el.attrs['href'] for el in bs.find_all("a", class_="clearfix trackable")])

https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=private


In [10]:
MAX_PER_PAGE = 35
pro_urls = []
num_pages = ceil(pro / 35)

for num in range(1, num_pages+1):
    particuliers_tab = base_url + "&owner_type=pro" + ("&page="+ str(num) if num > 1 else "")
    print(particuliers_tab)
    driver.get(particuliers_tab)
    bs = BeautifulSoup(driver.page_source, "html.parser")
    pro_urls.extend([el.attrs['href'] for el in bs.find_all("a", class_="clearfix trackable")])

https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=pro
https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=pro&page=2
https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=pro&page=3
https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=pro&page=4
https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=pro&page=5
https://www.leboncoin.fr/recherche/?category=2&text=zoe&regions=12&owner_type=pro&page=6


In [11]:
len(particuliers_urls), len(pro_urls)

(10, 201)

In [12]:
retrieved_phones = {}
car_datas = []

In [13]:
from lxml.html import fromstring
def get_free_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    print("found " + str(len(proxies)) + "proxies.")
    return proxies

In [14]:
def get_spys_proxies():
    b = driver.get("http://spys.one/en/https-ssl-proxy/")
    driver.find_element_by_xpath("//select[@name='xpp']/option[text()='500']").click()
    driver.find_element_by_xpath("//select[@name='xf2']/option[text()='SSL+']").click()
    driver.find_element_by_xpath("//select[@name='xf5']/option[text()='HTTP']").click()

    # Test if text contins a valid IP address
    def matchIP(text):
        return re.match("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", text) != None
    # Extract full proxy address
    def extractProxy(text):
        groups = re.search("(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})(.*)(:\d*)$", text)
        if groups:
            return groups.group(1) + groups.group(3)
        else:
            return None
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    new_proxies = [extractProxy(el.text) for el in soup.find_all("font", class_="spy14") if matchIP(el.text)]
    print("found " + str(len(new_proxies)) + "proxies.")
    return new_proxies

In [15]:
def get_proxyBroker_proxies(num, result_list):
    import asyncio
    from proxybroker import Broker
    
    async def show(proxies):
        while True:
            proxy = await proxies.get()
            if proxy is None: break
            result_list.append( str(proxy.host) + ":" + str(proxy.port))
            print('Found proxy: %s' % proxy)
            
    proxies = asyncio.Queue()
    broker = Broker(proxies)
    tasks = asyncio.gather(
        broker.find(types=['HTTPS'], limit=num),
        show(proxies))
    
    loop = asyncio.get_event_loop()
#    loop.run_until_complete(tasks)

In [16]:
class ProxiesPool:
    def __init__(self, proxies):
        assert len(proxies) > 1, "Proxies is empty"
        self.proxies = []
        self.proxies.extend(proxies)
        self.proxies.append("")
        self.proxy = self.nextProxy()
    
    def getActiveProxy(self):
        return self.proxy
    
    def nextProxy(self):
        if len(self.proxies) > 1:
            self.proxy = self.proxies.pop()
            if not self.testProxy(self.proxy):
                return self.nextProxy()
        else:
            self.proxy = None
        return self.proxy
        
    def testProxy(self, proxy):
        try:
            response = requests.get('https://httpbin.org/ip',proxies={"http": proxy, "https": proxy}, verify=False, timeout=10)
            if response.status_code == 200:
                print("Proxy "+ proxy + " is valid.")
                return True
            else:
                print("Proxy "+ proxy + " has problems: " + str(response))
                return False
        except Exception as e:
            traceback.print_exc()
            print("Proxy "+ proxy + " has problems: " + str(e))
            return False

In [17]:
def getPhoneNumberWithProxy(item_id, proxy):
    if item_id in retrieved_phones:
        return retrieved_phones[item_id]
    url = "https://api.leboncoin.fr/api/utils/phonenumber.json"
    #url = "http://api.scraperapi.com/"
    #proxy_key="d5e14ef3eeabf7a5515cd1b4ce08e7d9"
    proxies={"http": proxy, "https": proxy}
    datas = {
      "list_id": item_id,
      "app_id": "leboncoin_web_utils",
      "key": "54bb0281238b45a03f0ee695f73e704f",
      "text": 1
    }
    headers = {
      "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36",
      "Accept-Language": "en-US,en;q=0.8,fr;q=0.6",
      "Referer": "https://www.leboncoin.fr/voitures/" + item_id + ".htm?ca=12_s",
      "Origin": "https://www.leboncoin.fr",
      "Accept-Encoding": "gzip, deflate, br"
    }
    try:
        print("ask number for item: " + item_id)
        resp = requests.post(url=url, data=datas, headers=headers, proxies=proxies)
        if resp.status_code == 200:
            utils = resp.json()['utils']
            if utils['status'] == "OK":
                retrieved_phones[item_id] = utils['phonenumber']
                print("phone retrieved successfully: " + utils['phonenumber'])
                return utils['phonenumber']
            else:
                print("Failed to get phone: " + resp.text)
                return None
    except:
        print("Failed to get phone number, request failed.")
        return None

In [ ]:
proxies = []
get_proxyBroker_proxies(200, proxies)

In [ ]:
from itertools import cycle
import traceback

#proxies = get_spys_proxies()
proxiesPool = ProxiesPool(proxies)

url = 'https://httpbin.org/ip'
#Get a proxy from the pool
proxy = proxiesPool.nextProxy()
for i in range(1,2):
    proxy = proxiesPool.getActiveProxy()
    if not proxy:
        print("No more availible proxies")
        break
    print("Request #" + str(i) + ": " + proxy)
    try:
#        response = requests.get(url,proxies={"http": proxy, "https": proxy})
        response = getPhoneNumberWithProxy("1492326333", proxy)
        print(proxy + ": " + str(response))
    except Exception as e:
        traceback.print_exc()
        proxy = proxiesPool.nextProxy()
        #Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work. 
        #We will just skip retries as its beyond the scope of this tutorial and we are only downloading a single url 
        print("Skipping. Connnection error: " + str(e))


In [25]:
def getCarData(car_url, proxies_pool):
    search_rs = re.search('\/voitures\/(\d*).htm\/', car_url)
    if not search_rs:
        print("url " + car_url + " not clean")
        return None
    item_id = search_rs.group(1)
    car_data = {}
    car_data['id'] = item_id
    base_url = "https://api.leboncoin.fr/finder/classified/"
    url = base_url + item_id
    headers = {
        "api_key": "ba0c2dad52b3ec",
        "origin": "https://www.leboncoin.fr"
    }
    resp = requests.get(url, headers=headers)
    json_resp = resp.json()
    car_data['price'] = float(json_resp['price'][-1])
    for attr in json_resp['attributes']:
        if attr['key'] == 'model':
            car_data['model'] = attr['value_label']
        if attr['key'] == 'regdate':
            car_data['year'] = int(attr['value_label'])
        if attr['key'] == 'mileage':
            mileage_group = re.search('(\d*) km', attr['value_label'])
            if mileage_group:
                car_data['mileage'] = float(mileage_group.group(1))
            else:
                car_data['mileage'] = attr['value_label']
    car_data['phone'] = None
    if json_resp['has_phone']:
        phone = None
        proxy = proxies_pool.getActiveProxy()
        if proxy != None:
            while phone is None and proxy != None:
                phone = getPhoneNumberWithProxy(item_id, proxy)
                if phone:
                    break
                else:
                    print("failed to get phone for " + item_id + " with proxy " + proxy)
                    proxy = proxies_pool.nextProxy()
        else:
            print("No more proxies availible, setting phone to None.")
        car_data['phone'] = phone
    return car_data

<input>:2: DeprecationWarning: invalid escape sequence \/
<input>:24: DeprecationWarning: invalid escape sequence \d
<ipython-input-25-e81cfa7c6c1a>:2: DeprecationWarning: invalid escape sequence \/
  search_rs = re.search('\/voitures\/(\d*).htm\/', car_url)
<ipython-input-25-e81cfa7c6c1a>:24: DeprecationWarning: invalid escape sequence \d
  mileage_group = re.search('(\d*) km', attr['value_label'])


In [ ]:
#proxies = get_free_proxies()
proxies_pool = ProxiesPool(proxies)

In [ ]:

car_datas = []
for pro_url in pro_urls:
    car_data = getCarData(pro_url, proxies_pool)
    if car_data:
        car_data['type'] = "pro"
        car_datas.append(car_data)

for particuliers_url in particuliers_urls:
    car_data = getCarData(pro_url, proxies_pool)
    if car_data:
        car_data['type'] = "private"
        car_datas.append(car_data)

In [27]:
df = pd.DataFrame(car_datas)
df.head()

,id,mileage,model,phone,price,type,year
0,1513744824,7828.0,Zoe,0186655016,13990.0,pro,2017
1,1513249384,26578.0,Zoe,0160059293,9990.0,pro,2015
2,1513249395,37535.0,Zoe,0160059293,9990.0,pro,2015
3,1513183493,31200.0,Zoe,0176381010,8900.0,pro,2013
4,1511037307,55000.0,Zoe,0143084001,7490.0,pro,2015


In [34]:
df[["id", "phone"]].count()

id       211
phone     40
dtype: int64

We got only 40 phone number out of 211